In [ ]:
!nvcc --version

In [ ]:
!pip install pytorch_fid
!pip install mxnet-cu110
!git clone https://github.com/cheese-roll/light-anime-face-detector

In [ ]:
%cd light-anime-face-detector

In [ ]:
LAFD_PATH    = 'light-anime-face-detector'
CONFIG_PATH  = LAFD_PATH + '/configs/anime.json'
DATASET_PATH = 'anime'

In [ ]:
import sys
sys.path.append(LAFD_PATH)

import os
import torch
from pytorch_fid.fid_score import compute_statistics_of_path
from pytorch_fid.fid_score import calculate_frechet_distance
from pytorch_fid.inception import InceptionV3

import cv2
import json
from tqdm import tqdm
from core.detector import LFFDDetector


class Evaluator:
    def __init__(self, dataset_path, batch_size=64, dims=2048):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dataset_path = dataset_path

        assert dims in InceptionV3.BLOCK_INDEX_BY_DIM
        block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
        model = InceptionV3([block_idx]).to(device)

        self.params = {
            'model': model,
            'batch_size': batch_size,
            'dims': dims,
            'device': device,
        }
        self.m1, self.s1 = compute_statistics_of_path(self.dataset_path, **self.params)

        with open(CONFIG_PATH, "r") as f:
            config = json.load(f)
        self.detector = LFFDDetector(config, use_gpu=True)
        
    def FID(self, generated_path):
        m, s = compute_statistics_of_path(generated_path, **self.params)
        return calculate_frechet_distance(self.m1, self.s1, m, s)

    def AFD(self, generated_path):
        image_files = os.listdir(generated_path)
        
        count = 0
        for image_file in tqdm(image_files):
            image_path = os.path.join(generated_path, image_file)
            img = cv2.imread(image_path)
            boxes = self.detector.detect(img)
            count += 1 if boxes else 0

        return (count / len(image_files))

Eval = Evaluator(DATASET_PATH)

In [ ]:
def final_score(fid_score, afd_rate):
    standards = {
        'Default': (150.0, 0.60, 0),
        'Simple':  (120.0, 0.70, 4),
        'Normal':  (100.0, 0.80, 4),
        'Medium':  ( 90.0, 0.85, 4),
        'Hard':    ( 80.0, 0.90, 4),
        'Boss':    ( 70.0, 0.95, 4),
    }

    score = 0
    for title, standard in standards.items():
        if fid_score <= standard[0] and afd_rate >= standard[1]:
            score += standard[2]

    return score

In [ ]:
path = 'generated' # Path to your generated images
fid_score = Eval.FID(path)
afd_rate  = Eval.AFD(path)

print(f'FID score: {fid_score:.2f}')
print(f'AFD rate:  {(afd_rate*100):.2f}%')
print(f'Final score: {final_score(fid_score, afd_rate)}/20')